In [10]:
import torch
x = torch.Tensor(5, 3)
print x

x = torch.rand(5,3)
print x

n = x.numpy()
print n

t = torch.from_numpy(n)
print t

c = t.cuda()
print c


 1.5005e-37  0.0000e+00 -7.6662e+02
 4.5562e-41 -7.6663e+02  4.5562e-41
 3.2602e-38  0.0000e+00  3.2603e-38
 0.0000e+00  7.1108e+37  4.5562e-41
-2.0679e+07  4.5562e-41  0.0000e+00
[torch.FloatTensor of size 5x3]


 0.4897  0.4298  0.3249
 0.4832  0.4871  0.8124
 0.8364  0.5842  0.3870
 0.2969  0.0226  0.2087
 0.5753  0.5417  0.4954
[torch.FloatTensor of size 5x3]

[[ 0.48969743  0.42983857  0.32488254]
 [ 0.48317397  0.48713782  0.81241864]
 [ 0.83640146  0.58417422  0.38703355]
 [ 0.29691932  0.02264219  0.20869948]
 [ 0.57528991  0.54174018  0.49538466]]

 0.4897  0.4298  0.3249
 0.4832  0.4871  0.8124
 0.8364  0.5842  0.3870
 0.2969  0.0226  0.2087
 0.5753  0.5417  0.4954
[torch.FloatTensor of size 5x3]




 0.4897  0.4298  0.3249
 0.4832  0.4871  0.8124
 0.8364  0.5842  0.3870
 0.2969  0.0226  0.2087
 0.5753  0.5417  0.4954
[torch.cuda.FloatTensor of size 5x3 (GPU 0)]



In [23]:
from torch.autograd import Variable

x1 = Variable(t, requires_grad=True)
y = x1 * 5
z = y.mean()
grad = torch.ones(1)*3
z.backward(grad)

print x1.grad

Variable containing:
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
[torch.FloatTensor of size 5x3]



In [40]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [40]:
import numpy as np
params = list(net.parameters())
len(params)
conv1 = params[0]
type(conv1)
np_conv1 = conv1.data.numpy()
print(conv1[0])
print(np_conv1[0])

Variable containing:
(0 ,.,.) = 
 -0.1556  0.0840 -0.0577 -0.0516 -0.1619
  0.1903  0.0136 -0.0816 -0.1010 -0.0973
  0.1047  0.1158 -0.0016 -0.0184 -0.1950
 -0.1638 -0.1161 -0.0367  0.1942 -0.0288
 -0.1451  0.1651 -0.1666  0.0855  0.0575
[torch.FloatTensor of size 1x5x5]

[[[-0.15563506  0.08402465 -0.0576573  -0.05163998 -0.16187267]
  [ 0.19033045  0.01363554 -0.08159894 -0.10103304 -0.09726713]
  [ 0.10465418  0.11576439 -0.00160195 -0.01835244 -0.19499579]
  [-0.16383953 -0.11605299 -0.0366952   0.19416147 -0.0288285 ]
  [-0.14513239  0.16513522 -0.1665758   0.08552103  0.0574772 ]]]


In [46]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)

target = Variable(torch.arange(0, 10))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(out, target)
print loss
#Sets gradients of all model parameters to zero.
net.zero_grad() 
out.backward(torch.ones(1, 10), retain_graph=True)

Variable containing:
 28.7692
[torch.FloatTensor of size 1]



In [47]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
 -0.9767
  5.9167
 -4.7758
  7.6609
  3.9083
  2.6372
[torch.FloatTensor of size 6]



In [48]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [49]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update